In [56]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#for converting countryshortname to continent
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

#to get longitude and latitude data from country name
from geopy.geocoders import Nominatim


#for creating a world map (User Distributions around the world)
import folium
from folium.plugins import MarkerCluster

In [57]:
#Load Data
df = pd.read_csv("C:\\Users\\12132\\Desktop\\John\\medium\\creating a world map\\country_data.csv", encoding='latin-1')

In [58]:
df.head()

,CountryName,User_Percent
0,United States,0.013078
1,China,0.010516
2,Germany,0.009537
3,United Kingdom,0.009471
4,Brazil,0.009379


In [6]:
#convert to alpha 2 and continent

In [59]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

def get_continent(col):
    try:
        cn_a2_code =  country_name_to_country_alpha2(col)
    except:
        cn_a2_code = 'Unknown' 
    try:
        cn_continent = country_alpha2_to_continent_code(cn_a2_code)
    except:
        cn_continent = 'Unknown' 
    return (cn_a2_code, cn_continent)

In [60]:
#get country and continent info
df['codes'] = df['CountryName'].apply(get_continent)

split = df['codes'].astype(str).str.strip('()').str.split(',', expand=True)
#replace aphostrophes and white spaces
df['Country'] =split[0].str.replace("'",'').replace("'",'').str.strip()
df['Continent'] =split[1].str.replace("'",'').replace("'",'').str.strip()

In [61]:
df.head()

,CountryName,User_Percent,codes,Country,Continent
0,United States,0.013078,"(US, NA)",US,NA
1,China,0.010516,"(CN, AS)",CN,AS
2,Germany,0.009537,"(DE, EU)",DE,EU
3,United Kingdom,0.009471,"(GB, EU)",GB,EU
4,Brazil,0.009379,"(BR, SA)",BR,SA


In [ ]:
#get coordinates

In [62]:
#to get longitude and latitude data from country name
from geopy.geocoders import Nominatim

#function to get Longitude and Latitude
geolocator = Nominatim()

def geolocate(country):
    try:
        # Geolocate the center of the country
        loc = geolocator.geocode(country)
        # And return latitude and longitude
        return (loc.latitude, loc.longitude)
    except:
        # Return missing value
        return np.nan

C:\Users\12132\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


In [63]:
df['Geolocate'] = df['Country'].apply(geolocate)

In [64]:
# split into longitude and latitude columns
split = df['Geolocate'].astype(str).str.strip('()').str.split(', ', expand=True)
df['Latitude'] =split[0].astype('float64')
df['Longitude'] =split[1].astype('float64')

# remove na
df = df[~df['Latitude'].isna()]

In [65]:
df.head()

,CountryName,User_Percent,codes,Country,Continent,Geolocate,Latitude,Longitude
0,United States,0.013078,"(US, NA)",US,NA,"(39.7837304, -100.4458825)",39.783730,-100.445882
1,China,0.010516,"(CN, AS)",CN,AS,"(35.000074, 104.999927)",35.000074,104.999927
2,Germany,0.009537,"(DE, EU)",DE,EU,"(51.0834196, 10.4234469)",51.083420,10.423447
3,United Kingdom,0.009471,"(GB, EU)",GB,EU,"(54.7023545, -3.2765753)",54.702354,-3.276575
4,Brazil,0.009379,"(BR, SA)",BR,SA,"(-10.3333333, -53.2)",-10.333333,-53.200000


In [ ]:
#create a map

In [67]:
# Create a world map to show distributions of users 
#empty map
world_map= folium.Map(tiles="cartodbpositron")

marker_cluster = MarkerCluster().add_to(world_map)

for i in range(len(df)):
        lat = df.iloc[i]['Latitude']
        long = df.iloc[i]['Longitude']
        radius=5
        popup_text = """Country : {}<br>
                    %of Users : {}<br>"""
        popup_text = popup_text.format(df.iloc[i]['Country'],
                                   df.iloc[i]['User_Percent']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)

#show the map
world_map